## __App de pronósticos:__ ***(grrrrrraaaaww GITGITGIT)***

_Por Erick Ruiz y Martín Cordón_

In [ ]:
import time
import datetime
import datetime as dt
import streamlit as st
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from openpyxl import Workbook
from operator import index
from io import BytesIO
from prophet import Prophet
from prophet.plot import plot_plotly

En el bloque anterior de código se hace la importación de las librerías que se utilizarán para correr el programa. 

In [ ]:
### Preliminares

def date(y, m, d):

    period = int(time.mktime(datetime.datetime(y, m, d, 23, 59).timetuple()))
    return period

#inicio = date(2000, 1, 1)
hoy = int(time.mktime(datetime.datetime.now().timetuple()))

Se define preliminarmente la variable "hoy". Esta variable será útil en la consulta que se realizará más adelante. 

Originalmente, se había definido la función date, la cual devuelve la fecha en formato _mktime_, dicho formato es el solicitado por _Yahoo Finance_ al momento de realizar consultas, sin embargo, la función se descartó pues se optó por una fecha de inicio dinámica.

In [ ]:
# Dashboard

st.title('App de pronósticos: Martín Cordón y Erick Ruíz -grrrrrraaaaww GITGITGIT')

inicio = st.date_input('Introduzca una fecha', datetime.datetime(2000, 1, 1, 23, 59))
inicio2 = int(time.mktime(inicio.timetuple()))

ticker = st.text_input('Introduzca el "ticker" del activo que desea analizar', "PZZA")

interval = st.selectbox('Escoja un intervalo de tiempo', ['1d', '1wk', '1mo'])

n_years = st.slider('Años de pronóstico:', 1, 4)

if interval == '1d':

    periodo = n_years * 365

elif interval == '1wk':

    periodo = n_years * 52

else:

    periodo = n_years * 12

Se optó por la librería _Streamlit_ para hacer la presentación visual de los datos consultados en _Yahoo Finance_. 

Se define el título de la aplicación, una fecha de inicio interactiva, una caja de texto para ingresar el _ticker_ del activo a analizar, un intervalo para los datos del análisis y una cantidad de años para el pronóstico subsecuente. 

In [ ]:
# Query

@st.cache
def fin_data(ticker: str):
    
    qstr = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={inicio2}&period2={hoy}&interval={interval}&events=history&includeAdjustedClose=true'

    datos = pd.read_csv(qstr)

    return datos

loadstate_datos = st.text('Cargando datos...')
data = fin_data(ticker)
loadstate_datos.text('Cargando datos... listo!!')

st.subheader('Datos crudos')
st.write(data.tail())

Se procede a realizar un _web scrap_ sencillo a través de un _query_ dinámico en _Yahoo Finance_, a partir del _ticker_, los periodos de tiempo y el intervalo de los datos definidos en los pasos anteriores, es posible obtener la información histórica de precios de cualquier activo presente en las bolsas de valores de Estados Unidos. 

Desde la función principal del _query_ se procede a guardar los datos obtenidos en formato _DataFrame_ de la librería _Pandas_. Posteriormente se presenta una tabla de los últimos 5 datos obtenidos.

In [ ]:
@st.cache
def xls_df(df):

    output = BytesIO()
    writer = pd.ExcelWriter(output, engine="xlsxwriter")
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    format1 = workbook.add_format({'num_format': '0.00'})
    worksheet.set_column('A:A', None, format1)
    writer.save()
    processed_data = output.getvalue()
    return processed_data

xlsx = xls_df(data)

st.download_button(label='Descarga los datos en formato Excel',
                   data=xlsx,
                   file_name=f'historical_prices_{ticker}.xlsx')

Consideramos que una persona interesada en buscar los datos históricos de un activo financiero también estaría interesada en manipularlos por lo que se crea una función a partir de la cual, es posible transformar los datos de formato _DataFrame_ a un archivo de Excel. Se ofrece al usuario la oportunidad de descargar los datos en dicho formato a través de un botón de descarga.

In [ ]:
# Candelas

def candle():

    fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                    open=data['Open'],
                    high=data['High'],
                    low=data['Low'],
                    close=data['Close'])])
    fig.layout.update(title_text=f'Gráfico de candelas interactivo para {ticker}', xaxis_rangeslider_visible=True)
    st.plotly_chart(fig)

candle()

Al mismo tiempo, consideramos que los usuarios podrían estar interesados en conocer inmediatamente la trayectoria de los activos de su interés, por lo que se ofrece también al usuario la posibilidad de visualizar los datos consultados en un gráfico de velas japonesas interactivo, sobre el mismo es posible hacer _zoom_ a periodos específicos en el tiempo y observar la apertura, cierre, altos y bajos para el intervalo de tiempo definido anteriormente.

In [ ]:
# Prophet

df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=periodo)
forecast = m.predict(future)

st.subheader('Datos pronosticados')
st.write(forecast.tail())

st.write(f'Pronóstico para {n_years} años')
fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

st.write("Componentes del pronóstico")
fig2 = m.plot_components(forecast)
st.write(fig2)

Por último, es muy probable que sea del interés del usuario hacerse una idea de la trayectoria futura del activo de su interés. A través de la librería _Prophet_ de _Facebook_ se ofrece un pronóstico para la trayectoria del precio con la cantidad de años definida en el encabezado. Por restricciones de tiempo se optó por un modelo sencillo, sin embargo, una de las grandes ventajas de la librería _Prophet_ es la profunidad de calibración de los modelos que ofrece la herramienta. 

A los futuros usuarios de nuestra aplicación se recomienda jugar con las opciones disponibles para obtener el modelo que mejor pueda describir la trayectoria de los activos de su interés. Vale la pena mencionar que nuestra herramienta también ofrece la descomposición de la serie de tiempo en sus componentes principales y permite observar la tendencia y estacionalidad en la trayectoria del activo según los datos consultados.